
![Image Description](/images/Operations_Research_Methodology_Diagram.png)







I've been working on a workforce scheduling problem using linear programming in Python with the PuLP library. I'm trying to assign employees to different brands, ensuring certain conditions are met. Here's a detailed explanation of the problem and the code I've implemented:

Problem Statement:
Assign a fixed number of employees to different brands.
Ensure that an employee works on only one brand per day.
An employee can work a maximum of 9 hours and a minimum of 5 hours consecutively.
Need to fulfill staffing requirements for each brand.

In [7]:
import itertools

from pulp import LpProblem, LpVariable, LpBinary, lpSum

n_weeks = 2
days_per_week = 7
total_days = n_weeks * days_per_week
shop_open = 10  # 10 AM
shop_close = 20  # 8 PM
min_working_hours = 5
max_working_hours = 9
n_employees = 25

### DATA

# 5-day or 4-day contract type (60% each for now)
five_days_count = n_employees * 0.6

brands = {
    "PHONE": ["PHONE1", "PHONE2"],
    "TV": ["TV1", "TV2", "TV3"]
}
brand_items = tuple(itertools.chain(*brands.values()))

# employee table  e# : ( 4/5 day , [brand_item, ...] )
employees = {f'e{i}': (5*n_weeks if i < five_days_count else 4*n_weeks, [brand_items[i % len(brand_items)]]) for i in range(n_employees)}
# print(employees)

staffing_requirements = {
    "PHONE1": 1,
    "PHONE2": 1,
    "TV1": 1,
    "TV2": 1,
    "TV3": 1}

# the hours on which it is possible to commence a 5-hr shift
poss_starts = [hr for hr in range(shop_open, shop_close - min_working_hours + 1)]

prob = LpProblem('shift_sked')

### SETS / INDEXES

hours = list(range(shop_open, shop_close))  # convenience for readability...
days = list(range(total_days))
#                                                     Domain trim-down:  vvvv                 vvvv
EDHB_employees = {(e, d, h, b) for e in employees for d in days for h in poss_starts for b in employees[e][1]}
EDHB_brand_items = {(e, d, h, b) for e in employees for d in days for h in hours for b in employees[e][1]}

### VARS

start_shift = LpVariable.dicts('start', indices=EDHB_employees, cat=LpBinary) # e starts shift on day d, hour h, for brand item b
covers = LpVariable.dicts('covers', indices=EDHB_brand_items, cat=LpBinary)  # e covers brand item b on day d, hour h
max_shifts = LpVariable('max_shifts')  # the max shifts by any employee

### OBJ:  Minimize shift starts
prob += lpSum(start_shift)

# alternate objectives for experimenting...
# prob += lpSum(covers)  # should produce days*hours*requirements
# prob += max_shifts  # the max number of shifts by any employee  NOTE:  This is tougher solve, longer...

### CONSTRAINTS

# 1 & 2.  Limit shift starts by 4/5 day limit, and can only start 1 shift/day
for e in employees:
    prob += sum(start_shift[e, d, h, b] for d in days for h in poss_starts for b in employees[e][1]) <= employees[e][0]
    for d in days:
        prob += sum(start_shift[e, d, h, b] for h in poss_starts for b in employees[e][1]) <= 1

# 3. Link coverage to shift starting
for (e, d, h, b) in EDHB_brand_items:
    if b not in employees[e][1]:  # this employee cannot 'cover' this item
        prob += covers[e, d, h, b] <= 0
    elif h in poss_starts:  # a start or continued coverage can work...
        prev_hour = covers[e, d, h-1, b] if h > shop_open else None
        prob += covers[e, d, h, b] <= start_shift[e, d, h, b] + prev_hour
    else:  # only previous hour coverage can work (start not possible)
        prob += covers[e, d, h, b] <= covers[e, d, h-1, b]

# 4.  min/max coverage
for e in employees:
    for d in days:
        starts_shift = sum(start_shift[e, d, h, b] for h in poss_starts for b in employees[e][1])
        prob += sum(covers[e, d, h, b] for h in hours for b in employees[e][1]) <= max_working_hours * starts_shift
        prob += sum(covers[e, d, h, b] for h in hours for b in employees[e][1]) >= min_working_hours * starts_shift

# 5.  brand-item coverage minimums
for (d, h, b) in itertools.product(days, hours, brand_items):
    prob += sum(covers[e, d, h, b] for e in employees if b in employees[e][1]) >= staffing_requirements[b]

# 6.  Capture max shifts
for e in employees:
    prob += max_shifts >= sum(start_shift[e, d, h, b] for d in days for h in poss_starts for b in employees[e][1])

# print(prob)

cbc_path = '/opt/homebrew/opt/cbc/bin/cbc'
solver = pulp.COIN_CMD(path=cbc_path)
res = prob.solve(solver)

# highs_path = '/opt/homebrew/bin/highs'
# solver_2 = pulp.HiGHS_CMD(path=highs_path)
# res = prob.solve(solver_2)

NameError: name 'pulp' is not defined

In [108]:
import pandas as pd
from datetime import datetime, timedelta, time
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, value

file_path = r'C:\Users\Alvaro\Documents\Facultad\MBZUAI\Internship\Etihad\Internship\Zonal Allocation\Automated allocations\Automated Allocation Excel v6 - corrected dates.xlsm'
# Load data
manpower_df = pd.read_excel(file_path, sheet_name='Manpower')
flights_df = pd.read_excel(file_path, sheet_name='Flights')

# Let's handle the NaN values
manpower_df['Main Certifications'] = manpower_df['Main Certifications'].apply(lambda x: [] if pd.isna(x) else x)
manpower_df['Cat-A Certifications'] = manpower_df['Cat-A Certifications'].apply(lambda x: [] if pd.isna(x) else x)


In [109]:
manpower_df

,Name,Type,Primary Bay Zone,Main Certifications,Cat-A Certifications,Shift Start,Shift End
0,A.OBAID,LE,QC,"A321V, A32V, B787, B787-10",[],2024-07-15 17:00:00,2024-07-16 05:00:00
1,A.SHUAIBI,LE,C,"A321V, A32V, B787, B787-10",[],2024-07-15 17:00:00,2024-07-16 05:00:00
2,SANDEEP,LE,B,"A321V, A32V, A33T, A350, A380, B773, B77F",[],2024-07-15 17:00:00,2024-07-16 05:00:00
3,AMIN S,E,B,"A321V, A32V, A350, B787, B787-10",[],2024-07-15 18:00:00,2024-07-16 04:00:00
4,CRAIG,E,C,"B787, B787-10",[],2024-07-15 18:00:00,2024-07-16 04:00:00
5,ZAHID,E,C,"A32V, B773, B77F, B787, B787-10",[],2024-07-15 18:00:00,2024-07-16 04:00:00
6,ADIL,E,B,"A321V, A32V, B787, B787-10",[],2024-07-15 17:30:00,2024-07-16 05:30:00
7,BIJOY,E,C,"A380, B773, B77F",[],2024-07-15 17:30:00,2024-07-16 05:30:00
8,MITSIS,E,A,"A321V, A32V, A33T, B773, B77F, B787, B787-10",[],2024-07-15 17:30:00,2024-07-16 05:30:00
9,NASEERUDDIN,E,C,"A321V, A32V, B787, B787-10",[],2024-07-15 17:30:00,2024-07-16 05:30:00


In [103]:
flights_df

,Aircraft code,Aircraft type,Bay,Bay Zone,Arrival time,Departure time,Ground Time (minutes),WP
0,AED,A321V,620,B,2024-07-15 18:00:00,2024-07-16 03:10:00,550.0,AED/L-150724-2
1,AEO,A321N,602,A,2024-07-15 18:35:00,2024-07-15 21:40:00,185.0,AEO/L-150724
2,EJA,A32V,601,A,2024-07-15 18:35:00,2024-07-15 21:00:00,145.0,EJA/L-150724-2
3,AES,A321N,622,B,2024-07-15 18:45:00,2024-07-15 22:30:00,225.0,AES/L-150724-2
4,AEB,A321V,624,B,2024-07-15 19:05:00,2024-07-15 21:10:00,125.0,AEB/L-150724-2
...,...,...,...,...,...,...,...,...
66,XWC,A350,640,C,2024-07-16 00:50:00,2024-07-16 02:45:00,115.0,XWC/L-150724-2
67,DDB,B77F,208,A,2024-07-16 04:00:00,2024-07-16 17:00:00,780.0,DDB/L-160724
68,DDF,B77F,207,A,2024-07-16 04:00:00,2024-07-16 06:10:00,130.0,DDF/L-160724
69,ETE,B773,639,C,2024-07-16 04:40:00,2024-07-16 08:50:00,250.0,ETE/L-160724


In [110]:
# Extract certifiers, aircraft, bays, and shifts from the data
certifiers = manpower_df['Name'].unique()
aircraft_wp = flights_df['WP'].unique()
bays = flights_df['Bay'].unique()

# Create dictionaries for certifier and aircraft information
certifier_info = manpower_df.set_index('Name').to_dict('index')
aircraft_info = flights_df.set_index('WP').to_dict('index')

# Define what a long stop is, in minutes 
long_stop = 5*60

In [111]:
certifier_info

{'A.OBAID': {'Type': 'LE',
  'Primary Bay Zone': 'QC',
  'Main Certifications': 'A321V, A32V, B787, B787-10',
  'Cat-A Certifications': [],
  'Shift Start': Timestamp('2024-07-15 17:00:00'),
  'Shift End': Timestamp('2024-07-16 05:00:00')},
 'A.SHUAIBI': {'Type': 'LE',
  'Primary Bay Zone': 'C',
  'Main Certifications': 'A321V, A32V, B787, B787-10',
  'Cat-A Certifications': [],
  'Shift Start': Timestamp('2024-07-15 17:00:00'),
  'Shift End': Timestamp('2024-07-16 05:00:00')},
 'SANDEEP': {'Type': 'LE',
  'Primary Bay Zone': 'B',
  'Main Certifications': 'A321V, A32V, A33T, A350, A380, B773, B77F',
  'Cat-A Certifications': [],
  'Shift Start': Timestamp('2024-07-15 17:00:00'),
  'Shift End': Timestamp('2024-07-16 05:00:00')},
 'AMIN S': {'Type': 'E',
  'Primary Bay Zone': 'B',
  'Main Certifications': 'A321V, A32V, A350, B787, B787-10',
  'Cat-A Certifications': [],
  'Shift Start': Timestamp('2024-07-15 18:00:00'),
  'Shift End': Timestamp('2024-07-16 04:00:00')},
 'CRAIG': {'Type':

In [106]:
aircraft_info

{'AED/L-150724-2': {'Aircraft code': 'AED',
  'Aircraft type': 'A321V',
  'Bay': 620,
  'Bay Zone': 'B',
  'Arrival time': Timestamp('2024-07-15 18:00:00'),
  'Departure time': Timestamp('2024-07-16 03:10:00'),
  'Ground Time (minutes)': 550.0000000011642},
 'AEO/L-150724': {'Aircraft code': 'AEO',
  'Aircraft type': 'A321N',
  'Bay': 602,
  'Bay Zone': 'A',
  'Arrival time': Timestamp('2024-07-15 18:35:00'),
  'Departure time': Timestamp('2024-07-15 21:40:00'),
  'Ground Time (minutes)': 184.9999999953434},
 'EJA/L-150724-2': {'Aircraft code': 'EJA',
  'Aircraft type': 'A32V',
  'Bay': 601,
  'Bay Zone': 'A',
  'Arrival time': Timestamp('2024-07-15 18:35:00'),
  'Departure time': Timestamp('2024-07-15 21:00:00'),
  'Ground Time (minutes)': 145.00000000116415},
 'AES/L-150724-2': {'Aircraft code': 'AES',
  'Aircraft type': 'A321N',
  'Bay': 622,
  'Bay Zone': 'B',
  'Arrival time': Timestamp('2024-07-15 18:45:00'),
  'Departure time': Timestamp('2024-07-15 22:30:00'),
  'Ground Time (m

In [107]:
certifier_info['A.OBAID']['Shift Start']

'15/07/24 17:00'

In [67]:
########################### COSTS ########################################

# Function to calculate movement cost (or time) between bays
# If they are of the same area (for instance, both are 600), I define the movement cost as 2 minutes per bay, with a maximum of 10 minutes
# If they are of different areas, I set the movement cost as 30 minutes
def calculate_movement_cost(bay1, bay2):
    if bay1 // 100 == bay2 // 100:
        return min(2 * abs(bay1 - bay2), 10)
    else:
        return 30
    
# Create a movement time matrix. This matrix will be for all the possible pairs of aircraft
movement_cost_matrix = {}
for a1 in aircraft_wp:
    for a2 in aircraft_wp:
        movement_cost_matrix[(a1, a2)] = calculate_movement_cost(aircraft_info[a1]['Bay'], aircraft_info[a2]['Bay'])

# We can also define a cost of changing bay zones.
def calculate_zone_change_cost(bay_zone_1, bay_zone_2):
    if bay_zone_1 != bay_zone_2:
        return 100
    else:
        return 0

# Cost of assigning someone to more than 5 aircrafts 
def calculate_certifier_cost(certifier, aircrafts):
    if len(aircrafts) > 5:
        return 200
    else:
        return 0

# Cost of assigning a Cat-A certifier instead of a Main certifier
def calculate_certification_cost(certifier, aircraft):
    if certifier in aircraft_info[aircraft]['Main Certifications']:
        return 0
    elif certifier in aircraft_info[aircraft]['Cat-A Certifications']:
        return 50
    
# Cost of assigning a shift leader
def calculate_shift_leader_cost(certifier):
    if certifier_info[certifier]['Type'] == 'LE':
        return 100
    else:
        return 0
    
# Cost of assigning a quality control certifier
def calculate_quality_control_cost(certifier):
    if certifier_info[certifier]['Primary Bay Zone'] == 'Quality Control':
        return 200
    else:
        return 0
    
# Increasing cost of assinging more aircrafts to the same certifier? Like, if they have 1 aircraft, the cost is 0, if they have 2, the cost is 10, if they have 3, the cost is 20, and so on

In [44]:
aircraft_info

{'DDF/L-060624-2': {'Aircraft code': 'DDF',
  'Aircraft type': 'B77F',
  'Bay': 206,
  'Bay Zone': 'A',
  'Arrival time': datetime.time(3, 30),
  'Departure time': datetime.time(14, 35),
  'Ground Time (minutes)': 664.9999999999999},
 'BLS/L-070624': {'Aircraft code': 'BLS',
  'Aircraft type': 'B787',
  'Bay': 609,
  'Bay Zone': 'A',
  'Arrival time': datetime.time(4, 40),
  'Departure time': datetime.time(9, 55),
  'Ground Time (minutes)': 314.99999999999994},
 'ETI/L-070624': {'Aircraft code': 'ETI',
  'Aircraft type': 'B773',
  'Bay': 610,
  'Bay Zone': 'A',
  'Arrival time': datetime.time(5, 5),
  'Departure time': datetime.time(9, 5),
  'Ground Time (minutes)': 240.0},
 'BMF/L-070624': {'Aircraft code': 'BMF',
  'Aircraft type': 'B787-10',
  'Bay': 640,
  'Bay Zone': 'C',
  'Arrival time': datetime.time(5, 50),
  'Departure time': datetime.time(10, 25),
  'Ground Time (minutes)': 275.0},
 'BLQ/L-070624': {'Aircraft code': 'BLQ',
  'Aircraft type': 'B787',
  'Bay': 625,
  'Bay Zone

In [41]:
certifier_info

{'A.OBAID': {'Type': 'LE',
  'Primary Bay Zone': 'QC',
  'Main Certifications': 'A321V, A32V, B787, B787-10',
  'Cat-A Certifications': [],
  'Shift Start': datetime.time(17, 0),
  'Shift End': datetime.time(5, 0)},
 'A.SHUAIBI': {'Type': 'LE',
  'Primary Bay Zone': 'C',
  'Main Certifications': 'A321V, A32V, B787, B787-10',
  'Cat-A Certifications': [],
  'Shift Start': datetime.time(17, 0),
  'Shift End': datetime.time(5, 0)},
 'SANDEEP': {'Type': 'LE',
  'Primary Bay Zone': 'B',
  'Main Certifications': 'A321V, A32V, A33T, A350, A380, B773, B77F',
  'Cat-A Certifications': [],
  'Shift Start': datetime.time(17, 0),
  'Shift End': datetime.time(5, 0)},
 'AMIN S': {'Type': 'E',
  'Primary Bay Zone': 'B',
  'Main Certifications': 'A321V, A32V, A350, B787, B787-10',
  'Cat-A Certifications': [],
  'Shift Start': datetime.time(18, 0),
  'Shift End': datetime.time(4, 0)},
 'CRAIG': {'Type': 'E',
  'Primary Bay Zone': 'C',
  'Main Certifications': 'B787, B787-10',
  'Cat-A Certifications':

In [42]:
movement_cost_matrix

{('DDF/L-060624-2', 'DDF/L-060624-2'): 0,
 ('DDF/L-060624-2', 'BLS/L-070624'): 30,
 ('DDF/L-060624-2', 'ETI/L-070624'): 30,
 ('DDF/L-060624-2', 'BMF/L-070624'): 30,
 ('DDF/L-060624-2', 'BLQ/L-070624'): 30,
 ('DDF/L-060624-2', 'ETS/L-070624'): 30,
 ('DDF/L-060624-2', 'BLG/L-070624'): 30,
 ('DDF/L-060624-2', 'BNE/L-070624'): 30,
 ('DDF/L-060624-2', 'BMJ/L-070624'): 30,
 ('DDF/L-060624-2', 'BMA/L-070624'): 30,
 ('DDF/L-060624-2', 'BLN/L-070624'): 30,
 ('DDF/L-060624-2', 'BLB/L-070624'): 30,
 ('DDF/L-060624-2', 'BNC/L-070624'): 30,
 ('DDF/L-060624-2', 'BLA/L-070624'): 30,
 ('DDF/L-060624-2', 'BLY/L-070624'): 30,
 ('DDF/L-060624-2', 'API/L-070624'): 30,
 ('BLS/L-070624', 'DDF/L-060624-2'): 30,
 ('BLS/L-070624', 'BLS/L-070624'): 0,
 ('BLS/L-070624', 'ETI/L-070624'): 2,
 ('BLS/L-070624', 'BMF/L-070624'): 10,
 ('BLS/L-070624', 'BLQ/L-070624'): 10,
 ('BLS/L-070624', 'ETS/L-070624'): 10,
 ('BLS/L-070624', 'BLG/L-070624'): 10,
 ('BLS/L-070624', 'BNE/L-070624'): 10,
 ('BLS/L-070624', 'BMJ/L-070624

In [99]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus


# Initialize the problem
problem = LpProblem("Aircraft_Assignment", LpMinimize)

# Define decision variables
assignment = LpVariable.dicts("Assign", (certifiers, aircraft_wp), 0, 1, cat='Binary')

# Auxiliary variables for movement
movement = LpVariable.dicts("Movement", (certifiers, aircraft_wp, aircraft_wp), 0, 1, cat='Binary')

# Objective function: Minimize movement time
problem += lpSum([
    movement_cost_matrix[(a1, a2)] * movement[c][a1][a2]
    for c in certifiers for a1 in aircraft_wp for a2 in aircraft_wp
])

# Constraints
# Each aircraft should be assigned to exactly one certifier
for a in aircraft_wp:
    problem += lpSum([assignment[c][a] for c in certifiers]) == 1

# Each certifier should be assigned to a maximum of 5 aircraft, if possible
for c in certifiers:
    problem += lpSum([assignment[c][a] for a in aircraft_wp]) <= 5

# Certifiers should be assigned to aircraft they have certification, as Main Certification or Cat-A certification. If they don't have it, they can't be assigned
for c in certifiers:
    for a in aircraft_wp:
        if (aircraft_info[a]['Aircraft type'] not in certifier_info[c]['Main Certifications'] or
            aircraft_info[a]['Aircraft type'] not in certifier_info[c]['Cat-A Certifications']):
            problem += assignment[c][a] == 0


# Add the constraint that certifiers should be assigned to aircraft in their shift range + 1 hour



# Certifiers should be assigned to a maximum of 2 aircraft with long layovers
for c in certifiers:
    problem += lpSum([assignment[c][a] for a in aircraft_wp if aircraft_info[a]['Ground Time (minutes)'] > long_stop]) <= 2

# Link movement variables with assignments
for c in certifiers:
    for a1 in aircraft_wp:
        for a2 in aircraft_wp:
            if a1 != a2:
                problem += movement[c][a1][a2] >= assignment[c][a1] + assignment[c][a2] - 1
                problem += movement[c][a1][a2] <= assignment[c][a1]
                problem += movement[c][a1][a2] <= assignment[c][a2]

# Solve the problem
problem.solve()

# Check the solution
print(f"Status: {LpStatus[problem.status]}")
for c in certifiers:
    for a in aircraft_wp:
        if assignment[c][a].varValue == 1:
            print(f"Certifier {c} is assigned to Aircraft {a}")

Status: Infeasible


In [100]:
# Check the solution status
status = LpStatus[problem.status]
print(f"Status: {status}")

if status == 'Infeasible':
    print("The problem is infeasible. Analyzing constraints...")

    # Iterate over constraints to find those that are not satisfied
    for name, constraint in problem.constraints.items():
        if constraint.value() > constraint.constant:
            print(f"Constraint {name} is not satisfied.")
            print(f" - Expression: {constraint}")
            print(f" - Left-hand side value: {constraint.value()}")
            print(f" - Right-hand side value: {constraint.constant}")


# Check the solution (if feasible or optimal)
if status in ['Optimal', 'Feasible']:
    for c in certifiers:
        for a in aircraft_wp:
            if assignment[c][a].varValue == 1:
                print(f"Certifier {c} is assigned to Aircraft {a}")

Status: Infeasible
The problem is infeasible. Analyzing constraints...
